**Problema:** A gestão ineficiente de estoque de produtos perecíveis (laticínios) gera prejuízos financeiros e desperdício de alimentos. O objetivo é prever a quantidade de produtos que serão vendidos para otimizar o estoque.

**Dataset:** "Dairy Goods Sales Dataset" obtido no Kaggle. Contém dados de fazendas, produtos, preços e vendas no período de 2019 a 2022.

**Objetivo:** Desenvolver um modelo preditivo para estimar a variável alvo `Quantity Sold (liters/kg)`.

**Tipo de Aprendizado:** Aprendizado Supervisionado -> **Regressão**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

sns.set(style="whitegrid")

df = pd.read_csv('dairy_dataset.csv')

df['Price per Unit'] = df['Price per Unit'] * 0.06
df['Price per Unit (sold)'] = df['Price per Unit (sold)'] * 0.06

### Preparação dos Dados

In [ ]:
print(df.isnull().sum().sum())

print(df.duplicated().sum())

print(df[['Quantity Sold (liters/kg)', 'Price per Unit']].describe())

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Expiration Date'] = pd.to_datetime(df['Expiration Date'])

df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['Day_of_Week'] = df['Date'].dt.dayofweek
df['Days_to_Expire'] = (df['Expiration Date'] - df['Date']).dt.days

size_map = {'Small': 0, 'Medium': 1, 'Large': 2}
df['Farm Size'] = df['Farm Size'].map(size_map)

cols_drop = ['Approx. Total Revenue(INR)', 'Total Value', 'Product ID', 
             'Date', 'Production Date', 'Expiration Date', 
             'Quantity in Stock (liters/kg)']
df_clean = df.drop(columns=cols_drop)

### Análise Exploratória de Dados (EDA)

In [ ]:
plt.figure(figsize=(8, 4))
sns.histplot(df_clean['Quantity Sold (liters/kg)'], kde=True, color='blue')
plt.title('Distribuição da Quantidade Vendida')
plt.xlabel('Quantidade (Litros/kg)')
plt.show()

plt.figure(figsize=(8, 4))
sns.scatterplot(x='Price per Unit', y='Quantity Sold (liters/kg)', data=df_clean, alpha=0.5)
plt.title('Relação: Preço Unitário x Quantidade Vendida')
plt.show()

plt.figure(figsize=(10, 8))
numeric_cols = df_clean.select_dtypes(include=['float64', 'int64']).columns
sns.heatmap(df_clean[numeric_cols].corr(), cmap='coolwarm', annot=False)
plt.title('Matriz de Correlação')
plt.show()



### 5. Modelagem e Avaliação
1.  **Regressão Linear:** Modelo base (baseline).
2.  **Random Forest Regressor:** Modelo ensemble robusto (com otimização de hiperparâmetros).

In [ ]:
X = df_clean.drop('Quantity Sold (liters/kg)', axis=1)
y = df_clean['Quantity Sold (liters/kg)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

categorical_features = ['Location', 'Product Name', 'Brand', 'Storage Condition', 
                        'Customer Location', 'Sales Channel']
numeric_features = ['Total Land Area (acres)', 'Number of Cows', 'Farm Size', 
                    'Quantity (liters/kg)', 'Price per Unit', 'Price per Unit (sold)', 
                    'Shelf Life (days)', 'Minimum Stock Threshold (liters/kg)', 
                    'Reorder Quantity (liters/kg)', 'Month', 'Day_of_Week', 
                    'Year', 'Days_to_Expire']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [ ]:
model_lr = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', LinearRegression())])

model_lr.fit(X_train, y_train)
y_pred_lr = model_lr.predict(X_test)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))

print(f"Modelo 1 - Regressão Linear RMSE: {rmse_lr:.2f}")

In [ ]:
model_rf = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', RandomForestRegressor(random_state=42))])

param_grid = {
    'regressor__n_estimators': [50, 100],
    'regressor__max_depth': [10, 20, None]
}

grid = GridSearchCV(model_rf, param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
grid.fit(X_train, y_train)

best_rf = grid.best_estimator_
y_pred_rf = best_rf.predict(X_test)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))

print(f"\nModelo 2 - Random Forest (Melhor) RMSE: {rmse_rf:.2f}")
print("Melhores Parâmetros:", grid.best_params_)

In [ ]:
joblib.dump(best_rf, 'modelo_final_laticinios.pkl')
print("Modelo final salvo com sucesso: modelo_final_laticinios.pkl")